<a href="https://colab.research.google.com/github/samuel-c-santos/download_landsat_sentinel/blob/main/Download_Sentinel_e_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instale as bibliotecas necessárias (se ainda não instalou)
!pip install geemap --quiet
!pip install pycrs --quie

In [ ]:
# Importe as bibliotecas necessárias
import ee
import geemap
import datetime
import zipfile
import os
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# Autenticação e inicialização
ee.Authenticate()
ee.Initialize(project='ee-samuelsantosambientalcourse') # Substitua pelo seu projeto no google cloud

# Função para upload e extração do shapefile
def upload_shapefile():
    print("Faça upload de um arquivo .zip com o shapefile...")

    if os.path.exists("shapefile"):
        for f in os.listdir("shapefile"):
            os.remove(os.path.join("shapefile", f))
    else:
        os.makedirs("shapefile")

    uploaded = files.upload()

    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall("shapefile")
            print("Shapefile extraído com sucesso.")
            break
    else:
        print("Nenhum arquivo .zip encontrado.")
        return None

    shp_files = [f for f in os.listdir("shapefile") if f.endswith(".shp")]
    if not shp_files:
        print("Arquivo .shp não encontrado.")
        return None

    shp_path = os.path.join("shapefile", shp_files[0])
    return geemap.shp_to_ee(shp_path)

# Função para obter a imagem Landsat
def get_landsat_image(region, year, semester):
    start = datetime.date(year, 1 if semester == 1 else 7, 1)
    end = datetime.date(year, 6 if semester == 1 else 12, 30)

    if year <= 2011:
        collection_id = "LANDSAT/LT05/C02/T1_L2"
        band_options = ['SR_B5', 'SR_B4', 'SR_B3']
    elif year <= 2013:
        collection_id = "LANDSAT/LE07/C02/T1_L2"
        band_options = ['SR_B3', 'SR_B2', 'SR_B1']
    elif year <= 2021:
        collection_id = "LANDSAT/LC08/C02/T1_L2"
        band_options = ['SR_B6', 'SR_B5', 'SR_B4']
    else:
        collection_id = "LANDSAT/LC09/C02/T1_L2"
        band_options = ['SR_B4', 'SR_B3', 'SR_B2']

    collection = ee.ImageCollection(collection_id) \
        .filterBounds(region) \
        .filterDate(str(start), str(end)) \
        .filter(ee.Filter.lt('CLOUD_COVER', 10))

    if collection.size().getInfo() == 0:
        print(f"⚠️ Nenhuma imagem disponível para {year}-{semester}.")
        return None, None

    image = collection.median().select(band_options)
    return image, band_options

# Execução principal
feature = upload_shapefile()
if feature:
    region = feature.geometry()

    # Widgets para ano e semestre
    ano = widgets.IntSlider(description='Ano', value=2015, min=1984, max=2025)
    semestre = widgets.ToggleButtons(
        options=[('1º Semestre', 1), ('2º Semestre', 2)],
        description='Semestre:'
    )
    display(ano, semestre)

    def visualizar(b):
        year = ano.value
        sem = semestre.value
        img, vis_bands = get_landsat_image(region, year, sem)
        if img is None:
            print("⚠️ Nenhuma imagem disponível.")
            return

        # Calcular região expandida dinamicamente
        bounds = region.bounds()
        scale_factor = 5
        coords = bounds.coordinates().get(0).getInfo()

        xs = [pt[0] for pt in coords]
        ys = [pt[1] for pt in coords]
        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)
        xmid = (xmin + xmax) / 2
        ymid = (ymin + ymax) / 2
        xrange = (xmax - xmin) * scale_factor / 2
        yrange = (ymax - ymin) * scale_factor / 2
        expanded_bounds = ee.Geometry.Rectangle([xmid - xrange, ymid - yrange,
                                                 xmid + xrange, ymid + yrange])

        # Gerar link de download
        url = img.clip(expanded_bounds).getDownloadURL({
            'scale': 30,
            'region': expanded_bounds,
            'format': 'GeoTIFF'
        })

        print(f"📥 Link para download da imagem Landsat {year} - Semestre {sem}:\n{url}")

    botao = widgets.Button(description='Gerar link de download Landsat')
    botao.on_click(visualizar)
    display(botao)


In [ ]:
import ee
import geemap
import datetime
import zipfile
import os
from google.colab import files
from IPython.display import display
import ipywidgets as widgets

# Autenticação no Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-samuelsantosambientalcourse') # Substitua pelo seu projeto no google cloud

# Upload do shapefile
def upload_shapefile():
    print("Faça upload de um arquivo .zip com o shapefile...")

    if os.path.exists("shapefile"):
        for f in os.listdir("shapefile"):
            os.remove(os.path.join("shapefile", f))
    else:
        os.makedirs("shapefile")

    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall("shapefile")
            print("Shapefile extraído com sucesso.")
            break
    else:
        print("Nenhum arquivo .zip encontrado.")
        return None

    shp_files = [f for f in os.listdir("shapefile") if f.endswith(".shp")]
    if not shp_files:
        print("Arquivo .shp não encontrado.")
        return None

    shp_path = os.path.join("shapefile", shp_files[0])
    return geemap.shp_to_ee(shp_path)

# Obter imagem Sentinel-2
def get_sentinel_image(region, year, start_month, end_month):
    if end_month < start_month:
        print("⚠️ Mês final deve ser maior ou igual ao mês inicial.")
        return None

    start_date = datetime.date(year, start_month, 1)
    end_day = 28 if end_month == 2 else 30
    end_date = datetime.date(year, end_month, end_day)

    collection = (ee.ImageCollection("COPERNICUS/S2_SR")
                  .filterBounds(region)
                  .filterDate(str(start_date), str(end_date))
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))

    if collection.size().getInfo() == 0:
        print(f"Nenhuma imagem disponível para {year}/{start_month}-{end_month} com menos de 10% de nuvens.")
        return None

    return collection.median().select(['B4', 'B3', 'B2'])  # RGB

# Execução principal
feature = upload_shapefile()
if feature:
    region = feature.geometry()

    # Expandir região (sem visualização)
    bounds = region.bounds()
    scale_factor = 5
    coords = bounds.coordinates().get(0).getInfo()
    xs = [pt[0] for pt in coords]
    ys = [pt[1] for pt in coords]
    xmid = (min(xs) + max(xs)) / 2
    ymid = (min(ys) + max(ys)) / 2
    xrange = (max(xs) - min(xs)) * scale_factor / 2
    yrange = (max(ys) - min(ys)) * scale_factor / 2
    expanded_bounds = ee.Geometry.Rectangle([xmid - xrange, ymid - yrange,
                                             xmid + xrange, ymid + yrange])

    # Widgets
    current_year = datetime.datetime.now().year
    ano = widgets.IntSlider(description='Ano', value=current_year, min=2015, max=current_year)
    mes_inicio = widgets.Dropdown(description='Mês início', options=list(range(1, 13)), value=6)
    mes_fim = widgets.Dropdown(description='Mês fim', options=list(range(1, 13)), value=8)
    display(ano, mes_inicio, mes_fim)

    def gerar_link(b):
        img = get_sentinel_image(region, ano.value, mes_inicio.value, mes_fim.value)
        if img is None:
            return
        url = img.clip(expanded_bounds).getDownloadURL({
            'scale': 10,
            'region': expanded_bounds,
            'format': 'GeoTIFF'
        })
        print(f"📥 Link para download da imagem Sentinel-2:\n{url}")

    botao = widgets.Button(description='Gerar link Sentinel-2')
    botao.on_click(gerar_link)
    display(botao)
